# `cell2location` deconvolution of SlideSeq data  --muscle dataset 

Link to vignette:
https://github.com/BayraktarLab/cell2location/blob/master/docs/notebooks/cell2location_short_demo.ipynb
**First part of the cell2location pipeline//Here we construct the reference needed for deconvolution
This only need to be run once per tissue type/reference

In [ ]:
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import os
import seaborn as sns
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import gc
gc.enable()
import torch

In [ ]:
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=float32,force_device=True'
import cell2location

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:8000"

In [ ]:
#### TODO Future add this to utils 

def add_spatial_coordinates(adata, csv_path):
    # Read the CSV file containing cell names and coordinates
    spatial_df = pd.read_csv(csv_path, sep='\t', index_col=0, names=['x', 'y'])

    # Merge the spatial information with the original AnnData object
    adata_spatial = pd.merge(adata.obs, spatial_df, left_index=True, right_index=True, how='left')

    # Add NaN for cells without coordinates
    adata_spatial[['x', 'y']] = adata_spatial[['x', 'y']].where(pd.notna(adata_spatial[['x', 'y']]), np.nan)

    # Print a message about the number of cells with added coordinates
    print(f"Added spatial coordinates for {adata_spatial[['x', 'y']].count().min()} cells.")

    # Add the 'spatial' component to the AnnData object
    adata.obsm['spatial'] = adata_spatial[['x', 'y']].values

    return adata

## Settings

Load data

# Run `cell2loc`

# Generate single-cell reference model

## Deconvolution section


In [ ]:
adata_ref = sc.read_h5ad("/workdir/in68/Muscle/sc_reference/20240216_AllDays_LDW_Aging.h5ad")

In [ ]:
adata_ref.X = adata_ref.raw.X.copy()
del adata_ref.raw
adata_ref.X


In [ ]:
#round raw counts because soupX yields non-integers and cell2loc is unhappy
from scipy.sparse import csr_matrix
adata_ref.X = csr_matrix(np.round(adata_ref.X).astype(np.int32))


In [ ]:
np.unique(adata_ref.obs['Specific_cell_types2'])

In [ ]:
adata_ref = adata_ref[adata_ref.obs['Specific_cell_types2'] != 'MuSCs and progenitors']
adata_ref = adata_ref[adata_ref.obs['Specific_cell_types2'] != 'Endothelial and Myeloid cells']

np.unique(adata_ref.obs['Specific_cell_types2'])

In [ ]:
cell_type_mapping = {
        'B cells' : 'B cells',
        'Committing MPCs' : 'MuSCs' ,
        'Cycling MPCs' : 'MuSCs',
        'Dendritic cells (Cd209a+)' : 'Dendritic cells',
        'Dendritic cells (Cd72+)': 'Dendritic cells',
        'Dendritic cells (Fscn1+)': 'Dendritic cells', 
        'Dendritic cells (Xcr1+)': 'Dendritic cells',
        'Endothelial cells (Artery)': 'Endothelial cells',
        'Endothelial cells (Capillary)': 'Endothelial cells',
        'Endothelial cells (Vein)': 'Endothelial cells',
        'Erythrocytes' : 'Erythrocytes',
        'FAPs (Adipogenic)': 'FAPs',
        'FAPs (Pro-remodeling)': 'FAPs',
        'FAPs (Stem)': 'FAPs', 
        'Fusing Myocytes': 'Fusing Myocytes',
        'M1 Macrophages (Ccr2+)' : 'Monocytes/Macrophages',
        'M1/M2 Macrophages (Mrc1+)': 'Monocytes/Macrophages', 
        'M2 Macrophages (Cx3cr1+)': 'Monocytes/Macrophages',
        'Monocytes (Cycling; Cdk1+)': 'Monocytes/Macrophages', 
        'Monocytes/Macrophages (Cxcl10+)': 'Monocytes/Macrophages',
        'Monocytes/Macrophages (Patrolling; Ctsa+)': 'Monocytes/Macrophages',
        'MuSCs 1':  'MuSCs',
        'MuSCs 2': 'MuSCs',
        'MuSCs 3': 'MuSCs',
        'MuSCs 4': 'MuSCs',
        'MuSCs 5': 'MuSCs',
        'MuSCs 6': 'MuSCs',
        'Myonuclei' : 'Myonuclei',
        'NK cells' : 'NK cells', 
        'Neutrophils' :  'Neutrophils',
        'Non-Cycling MPCs' : 'MuSCs',
        'Pericytes and Smooth muscle cells' : 'Pericytes and Smooth muscle cells',
        'Schwann and Neural/Glial cells' : 'Schwann and Neural/Glial cells', 
        'T cells (Cd4+)': 'T cells',
        'T cells (Cycling; Cd3e+)': 'T cells',
        'T cells (Non-cycling; Cd3e+)': 'T cells',
        'Tenocytes': 'Tenocytes'
}

adata_ref.obs['Cell_type'] = adata_ref.obs['Specific_cell_types2'].map(cell_type_mapping)

In [ ]:
adata_ref.obs['Cell_type']

In [ ]:
from cell2location.utils.filtering import filter_genes
###Standard default vals for heart reference// May requires change for bigger refs
#for adata in ref_dict.values:
selected = filter_genes(
        adata_ref,
        cell_count_cutoff=5,
        cell_percentage_cutoff2=0.01,
        nonz_mean_cutoff=1.12)
selected


In [ ]:
# filter the object
adata_ref = adata_ref[:, selected].copy()

In [ ]:
# prepare anndata for the regression model
cell2location.models.RegressionModel.setup_anndata(
    adata=adata_ref, 
        
    # 10X reaction / sample / batch
    batch_key='orig.ident', 
    
    # cell type, covariate used for constructing signatures
    labels_key='Cell_type'
    
    # multiplicative technical effects (platform, 3' vs 5', donor effect)
    # categorical_covariate_keys=['Method']
)

In [ ]:
# create the regression model
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref) 

# view anndata_setup as a sanity check
mod.view_anndata_setup()

In [ ]:
adata_ref

In [ ]:
mod.train(max_epochs=250, use_gpu=1)

In [ ]:
mod.plot_history(1)

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, 
    sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
)

# Save model
ref_run_name = "Muscle_ref_cell2loc_celltype2_040324"
mod.save(f"/fs/cbsuvlaminck2/workdir/in68/c2l/{ref_run_name}", overwrite=True)

# Save anndata object with results
adata_ref.write( f"/fs/cbsuvlaminck2/workdir/in68/c2l/{ref_run_name}.h5ad")

In [ ]:
# Load reference
ref_run_name = "Muscle_ref_cell2loc_celltype2_022924"
adata_ref = sc.read_h5ad(f"/fs/cbsuvlaminck2/workdir/in68/c2l/{ref_run_name}.h5ad")
mod = cell2location.models.RegressionModel.load(f"/fs/cbsuvlaminck2/workdir/in68/c2l/{ref_run_name}", adata_ref,use_gpu= True)

In [ ]:
mod.plot_QC()

export estimated expression in each cluster

In [ ]:
# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5,20:30 ]

In [ ]:
inf_aver.loc['Pax7']

In [ ]:
# adata_obj=[]
# adata_1 = sc.read_10x_mtx('/fs/cbsuvlaminck2/workdir/in68/Curio/align_out/Sample11/STARsolo/Solo.out/GeneFull/raw/')
# adata_1 = add_spatial_coordinates(adata_1,
#                                             f"/fs/cbsuvlaminck2/workdir/in68/Utils/Curio_BB/A0018_011_BeadBarcodes.txt")
    
# adata_1.raw = adata_1.copy()

# adata_2 = sc.read_10x_mtx('/fs/cbsuvlaminck2/workdir/in68/Curio/align_out/Sample12/STARsolo/Solo.out/GeneFull/raw/')

# adata_2 = add_spatial_coordinates(adata_2,
#                                             f"/fs/cbsuvlaminck2/workdir/in68/Utils/Curio_BB/A0018_012_BeadBarcodes.txt")
    
# adata_2.raw = adata_2.copy()







# adata_obj = ad.concat(adatas = [adata_1,adata_2], keys=['Y','G'],label = 'batch')

In [ ]:
#adata_obj = sc.read_10x_mtx('/fs/cbsuvlaminck2/workdir/in68/Curio/align_out/Sample11/STARsolo/Solo.out/GeneFull/raw/')
adata_obj = sc.read_h5ad("/workdir/in68/Muscle/out/h5ad/binned25_Sample11.h5ad")

In [ ]:
# adata_obj = add_spatial_coordinates(adata_obj,
#                                             f"/fs/cbsuvlaminck2/workdir/in68/Utils/Curio_BB/A0018_011_BeadBarcodes.txt")
    
#adata_obj.raw = adata_obj.copy()
min_counts = 20
adata_obj.var["mito"] = adata_obj.var_names.str.startswith("Mt-")
#adata_obj.obsm['MT'] = adata_obj[:, adata_obj.var['mito'].values].X.toarray()
adata_obj = adata_obj[:, ~adata_obj.var['mito'].values]
sc.pp.filter_cells(adata_obj, min_counts = min_counts)

In [ ]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_obj.var_names, inf_aver.index)
adata_obj = adata_obj[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_obj)

In [ ]:

cell2location.models.Cell2location.setup_anndata(adata=adata_obj)
# create and train the model
mod = cell2location.models.Cell2location(
    adata_obj, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=4,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=20
    
)
mod.view_anndata_setup()

In [ ]:
mod

In [ ]:
gc.collect()
mod.train(max_epochs=20000,
          # train using full data (batch_size=None)
          batch_size=6000,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu= 1
         )


In [ ]:
!nvidia-smi

In [ ]:
# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(100)
plt.legend(labels=['full data training']);

In [ ]:
mod

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
sample_name = 'Muscle_0404_Y'

adata = mod.export_posterior(
    adata_obj, sample_kwargs={'num_samples': 500, 'batch_size': 5000, 'use_gpu': True}
)

# Save model
#mod.save(f"{sample_name}", overwrite=True)
mod.save(f"/workdir/in68/Muscle/c2l{sample_name}", overwrite = True)



# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

# Save anndata object with results
adata_file = f"/workdir/in68/Muscle/{sample_name}.h5ad"
adata.write(adata_file)
adata_file



# # In this section, we export the estimated cell abundance (summary of the posterior distribution).
# sample_name = 'Muscle_all_0329_G'

# adata = mod.export_posterior(
#     adata_G, sample_kwargs={'num_samples': 1000, 'batch_size': 3000, 'use_gpu': True}
# )

# # Save model
# #mod.save(f"{sample_name}", overwrite=True)
# mod.save(f"/workdir/in68/Muscle/c2l{sample_name}", overwrite = True)



# # mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

# # Save anndata object with results
# adata_file = f"/workdir/in68/Muscle/{sample_name}.h5ad"
# adata.write(adata_file)
# adata_file

In [ ]:
adata_obj

In [ ]:
mod.plot_QC()


In [ ]:
adata_obj.obsm["spatial"]


In [ ]:
#adata_obj_2 = sc.read_10x_mtx('/fs/cbsuvlaminck2/workdir/in68/Curio/align_out/Sample12/STARsolo/Solo.out/GeneFull/raw/')
adata_obj_2 = sc.read_h5ad("/workdir/in68/Muscle/out/h5ad/Sample12.h5ad")

In [ ]:
# adata_obj_2 = add_spatial_coordinates(adata_obj_2,
#                                             f"/fs/cbsuvlaminck2/workdir/in68/Utils/Curio_BB/A0018_012_BeadBarcodes.txt")
    
# adata_obj_2.raw = adata_obj_2.copy()
# min_counts = 20
adata_obj_2.var["mito"] = adata_obj_2.var_names.str.startswith("Mt-")

adata_obj_2 = adata_obj_2[:, ~adata_obj_2.var['mito'].values]
sc.pp.filter_cells(adata_obj_2, min_counts = min_counts)

In [ ]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_obj_2.var_names, inf_aver.index)
adata_obj_2 = adata_obj_2[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_obj_2)

In [ ]:
# create and train the model
mod = cell2location.models.Cell2location(
    adata_obj_2, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=4,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=20
)
mod.view_anndata_setup()

In [ ]:
gc.collect()
mod.train(max_epochs=5000,
          # train using full data (batch_size=None)
          batch_size=6000,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size= 1,
          use_gpu= 2
         )


In [ ]:
!nvidia-smi

In [ ]:
# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(100)
plt.legend(labels=['full data training']);

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
sample_name = 'Muscle_0404_G'
adata_obj_2 = mod.export_posterior(
    adata_obj_2,  sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': 1}
)

# Save model
#mod.save(f"{sample_name}", overwrite=True)
mod.save(f"/workdir/in68/Muscle/c2l{sample_name}", overwrite = True)



# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

# Save anndata object with results
adata_file = f"/workdir/in68/Muscle/{sample_name}.h5ad"
adata_obj_2.write(adata_file)
adata_file

In [ ]:
mod.plot_QC()


In [ ]:
#####Add Back Spatial location

adata_obj.obs[adata_obj.uns['mod']['factor_names']] = adata_raw.obsm['q05_cell_abundance_w_sf']



sc_cluster_order = ['FAPs (Adipogenic)', 'FAPs (Pro-remodeling)',
       'FAPs (Stem)', 'Fusing Myocytes', 'M1 Macrophages (Ccr2+)',
       'M1/M2 Macrophages (Mrc1+)', 'M2 Macrophages (Cx3cr1+)',
       'Monocytes (Cycling; Cdk1+)', 'Monocytes/Macrophages (Cxcl10+)',
       'Monocytes/Macrophages (Patrolling; Ctsa+)', 'MuSCs 1', 'MuSCs 2',
       'MuSCs 3', 'MuSCs 4', 'MuSCs 5', 'MuSCs 6', 'Myonuclei',
       'NK cells', 'Neutrophils', 'Non-Cycling MPCs',
       'Pericytes and Smooth muscle cells']


sc.settings.set_figure_params(dpi_save= 400, fontsize=7, figsize=(3.0,3.0), facecolor='white', frameon=True, transparent=True, format="pdf")
sc.pl.spatial(adata_obj, color=sc_cluster_order, ncols=7, frameon=False, cmap = 'magma', 
              size=1.5, scale_factor=1.0, hspace=0.1, wspace=0.1,spot_size= 20,
              # limit color scale at 99.2% quantile of cell abundance
              vmin=0, vmax='p99.2'
             )

In [ ]:
adata_obj_2
adata_obj_2.obs[adata_obj_2.uns['mod']['factor_names']] = adata_obj_2.obsm['q05_cell_abundance_w_sf']
sc.settings.set_figure_params(dpi_save= 400, fontsize=7, figsize=(3.0,3.0), facecolor='white', frameon=True, transparent=True, format="pdf")
sc.pl.spatial(adata_obj_2, color=['MuSCs','Fusing Myocytes','Myonuclei', 'T cells'], ncols=7, frameon=False, cmap = 'magma', 
              size=1.5, scale_factor=1.0, hspace=0.1, wspace=0.1,spot_size= 1,
              # limit color scale at 99.2% quantile of cell abundance
              vmin=0, vmax='p99.2'
             )

In [ ]:
adata_obj
adata_obj.obs[adata_obj.uns['mod']['factor_names']] = adata_obj.obsm['q05_cell_abundance_w_sf']
sc.settings.set_figure_params(dpi_save= 400, fontsize=7, figsize=(3.0,3.0), facecolor='white', frameon=True, transparent=True, format="pdf")
sc.pl.spatial(adata_obj, color=['Fusing Myocytes','Myonuclei','MuSCs','T cells'], ncols=7, frameon=False, cmap = 'magma', 
              size=1.5, scale_factor=1.0, hspace=0.1, wspace=0.1,spot_size= 1,
              # limit color scale at 99.2% quantile of cell abundance
              vmin=0, vmax='p99.2'
             )

In [ ]:
adata_obj.obsm['means_cell_abundance_w_sf']
sc.pp.neighbors(adata_obj,use_rep='means_cell_abundance_w_sf')
sc.tl.leiden(adata_obj,resolution= 0.2)

In [ ]:
adata_obj

In [ ]:
sc.pl.spatial(adata_obj, color='leiden', ncols=1, frameon=False, cmap = 'magma', 
              size=1.5, scale_factor=1.0, hspace=0.1, wspace=0.1,spot_size= 1,
              # limit color scale at 99.2% quantile of cell abundance
              vmin=0, vmax='p99.2'
             )